In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from pathlib import Path
import re
import gradio as gr

class ChatGenerator:
    def __init__(self):
        self.checkpoint = (Path.cwd()).parent / 'Llama-3.2-3B-Instruct' # Set the model checkpoint path
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint) # Load tokenizer
        self.model = AutoModelForCausalLM.from_pretrained(
            self.checkpoint,
            torch_dtype=torch.bfloat16,
            device_map="cuda"  # Automatically uses GPU if available
        )
        self.text_generator = transformers.pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            torch_dtype=torch.bfloat16,
            device_map="cuda"  # Auto-detect and use GPU
        )

    def generate_response(self, question: str):
        prompt = (
            "<|begin_of_text|>"
            "<|start_header_id|>user<|end_header_id|>"
            f"{question}"
            "<|eot_id|>"
            "<|start_header_id|>assistant<|end_header_id|>"
        )

        # Generate sequences
        sequences = self.text_generator(
            prompt,
            do_sample=True,
            top_k=2,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            truncation=True,
            max_length=6000,
        )

        # Extract and return generated responses
        for sequence in sequences:
            match = re.search(r'<\|end_header_id\|>\\n\\n(.*?)\'\}', str(sequence), re.DOTALL)
            response = match.group(1)  # Gets everything after the last '<|end_header_id|>'
        return sequences, response

# Usage example
chat_bot = ChatGenerator()
chat_history = ""


while True:
    question = input("Enter your question: ")
    if question.lower() == "exit":
        break
    sequences, response = chat_bot.generate_response(chat_history + question)
    chat_history += str(sequences)
    print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The main characters in the Disney movie "Frozen" are:\n\n1. **Elsa** (voiced by Idina Menzel): The elder sister of Anna, Elsa is the princess of Arendelle and has the magical power to create and control ice and snow. She is the main protagonist of the story.\n2. **Anna** (voiced by Kristen Bell): The younger sister of Elsa, Anna is a optimistic, energetic, and determined princess who sets out to find Elsa and convince her to return home and reverse the eternal winter that has been cast upon Arendelle.\n\nHowever, if you\'re asking about the main character of the movie, I would say that it\'s a bit of a debate among fans, but many would argue that the main character is actually **Elsa**. Her journey and struggles with her powers and her relationship with Anna drive the plot of the movie.\n\nWhat do you think?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Anna is the blonde character in the Disney movie "Frozen".


...

Olaf.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Olaf.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AttributeError: 'NoneType' object has no attribute 'group'